<h1 style="font-size:200%; font-family:cursive; color:white;">1. Import Required Libraries & Dataset</h1>

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import AutoModel, BertTokenizer

#specify GPU
device = torch.device("cuda")

### Check if Pytorch is using GPU

In [ ]:
torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.current_device(), torch.cuda.device(0), torch.cuda.get_device_name(0), device

In [2]:
df_videos = pd.read_csv("/app/AI-Module/Resources/Datasets/how2sign.csv", usecols=["translation", "id"])
#df_videos = pd.read_csv("C:/Users/48519558/Desktop/SignAI-ML/AI-Module/Resources/Datasets/how2sign.csv", usecols=["translation", "id"])
#df_videos = pd.read_csv("D:/SignAI-ML/AI-Module/Resources/Datasets/how2sign.csv", usecols=["translation", "id"])

In [4]:
# df_videos = pd.read_csv("D:/Sprinfil/Dataset/how2sign.csv")
# df_videos = df_videos.sort_values(["id"]).reset_index().drop(["index"], axis=1)
# df_videos.head()
# import ast
# points = df_videos["points"]
# for index, point in enumerate(points):
#     print(index)
#     points.at[index] = ast.literal_eval(point)
# type(points.iloc[0])
# df_videos["points"] = points
# df_videos.head()
# for index, item in enumerate(points):
#     item = np.array(item)
#     print(index, item.shape)
#     item_with_index = np.array((item, index), dtype=object)
#     points.iloc[index]=item
#     np.save(f"{index}.npy", item_with_index)
# df_videos["points"] = points
# df_videos.head()

In [3]:
puntos_folder = "/app/AI-Module/Resources/Datasets/Points"
#puntos_folder = "C:/Users/48519558/Desktop/SignAI-ML/AI-Module/Resources/Datasets/Points"
#puntos_folder = "D:/SignAI-ML/AI-Module/Resources/Datasets/Points"
files = [puntos_folder + "/" + file for file in os.listdir(puntos_folder)]
len(files)

500

In [4]:
def load_points(files):
    for file in files:
        item_with_index = np.load(file, allow_pickle=True)
        item = item_with_index[0].astype(np.float16)
        index = item_with_index[1]
        yield item, index

puntos_list = []
ids_list = []

for item, index in load_points(files):
    puntos_list.append(item)
    ids_list.append(index)

df_puntos = pd.DataFrame({
    'points': puntos_list,
    'id': ids_list
})
df_puntos = df_puntos.sort_values(["id"]).reset_index(drop=True)

In [5]:
df_videos = df_videos.merge(df_puntos, on="id", how="inner")

***

In [6]:
max_len = df_videos['points'].apply(lambda x: len(x)).max()
print(max_len)
df_videos.head()

2537


,translation,id,points
0,If you like to find more about my services you...,448,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0..."
1,Hello welcome my name is Julio Nutt and I am a...,210,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0..."
2,"All right, the drink we're about to make is ca...",228,"[[1.0, 0.0589, 1.0, 1.0, 0.7363, 0.0, 0.6997, ..."
3,"Okay, another thing that for me is important. ...",187,"[[1.0, 0.0515, 1.0, 1.0, 0.7144, 0.0, 0.6934, ..."
4,"All right, we're ready to start our fire. What...",99,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0..."


In [7]:
def add_padding(max_frames, point_series: pd.Series):
    for i in range(len(point_series)):
        current_length = len(point_series[i])
        if current_length < max_frames:
            padding = np.full(
                (max_frames - current_length, 2172), 
                -1,
                dtype=np.float16
            )
            padding[:, 3::4] = 0
            point_series[i] = np.concatenate((point_series[i], padding), axis=0)
    return point_series

In [8]:
df_videos['points'] = add_padding(max_len, df_videos['points'])

/tmp/ipykernel_288/2892455734.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  point_series[i] = np.concatenate((point_series[i], padding), axis=0)
/tmp/ipykernel_288/2892455734.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  point_series[i] = np.concatenate((point_series[i], padding), axis=0)
/tmp/ipykernel_288/2892455734.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  point_series[i] = np.concatenate((point_series[i], p

In [9]:
from sklearn.model_selection import train_test_split
X_df = df_videos['points'].to_frame()
y_df = df_videos['translation'].to_frame()
seed = 31991
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=(100/500), random_state=seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=(100/400), random_state=seed)

In [10]:
X_train.head(), y_train.head(), X_val.head(), y_val.head(), X_test.head(), y_test.head()

(                                                points
 496  [[0.7964, 0.0, 1.0, 1.0, 0.5186, 0.051, 0.8564...
 417  [[1.0, 0.0138, 1.0, 1.0, 0.7383, 0.0, 0.637, 1...
 263  [[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...
 335  [[1.0, 0.0387, 1.0, 1.0, 0.735, 0.0, 0.755, 1....
 23   [[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...,
                                            translation
 496  Hi! Let it drip off a little bit, put it on a ...
 417  I'm Ivan Madar. Skiing is done from your waste...
 263  Hi, this Sean Hobson, and right now we're work...
 335  Developing reading skills isn't it important t...
 23   We have 2 different receipts to show you a pea...,
                                                 points
 12   [[1.0, 0.7817, 0.7466, 1.0, 0.8823, 0.4182, 0....
 256  [[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...
 172  [[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...
 339  [[1.0, 0.0044, 1.0, 1.0, 0.699, 0.0, 0.586, 1....
 352  [[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.

In [11]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((300, 1), (300, 1), (100, 1), (100, 1), (100, 1), (100, 1))

In [ ]:
import gc
globals().pop("df_videos", None)
globals().pop("puntos_list", None)
globals().pop("ids_list", None)
globals().pop("df_puntos", None)
globals().pop("X_df", None)
globals().pop("item", None)
globals().pop("y_df", None)
gc.collect()


<h1 style="font-size:200%; font-family:cursive; color:white;">3. Import Bert - base- uncased</h1>

In [15]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [16]:
# tokenize and encode sequences in the training set
tokens_label_train = tokenizer.batch_encode_plus(
    y_train['translation'].tolist(),
    padding=True,
    truncation=True,
    return_tensors="pt"
)

# tokenize and encode sequences in the validation set
tokens_label_val = tokenizer.batch_encode_plus(
    y_val['translation'].tolist(),
    padding = True,
    truncation = True,
    return_tensors="pt"
)

# tokenize and encode sequences in the test set
tokens_label_test = tokenizer.batch_encode_plus(
    y_test['translation'].tolist(),
    padding = True,
    truncation = True,
    return_tensors="pt"
)

<u><h2 style="font-size:170%; font-family:cursive;">What is the maximum sequence length of the input?</h2></u>

<p style="font-size:150%; font-family:verdana;">The maximum sequence length of the input = 512</p>

<h1 style="font-size:200%; font-family:cursive; color:white;">5. List to Tensors</h1>

In [25]:
def create_attention_mask_from_points(seq_tensor: torch.Tensor):
    mask = []
    for video in seq_tensor:
        mask_video = []
        for frame in video:
            mask_frame = (frame[..., :3] != -1).all(dim=-1) & (frame[..., 3] != 0)
            mask_video.append(mask_frame)
        mask.append(mask_video)
    mask = torch.stack([torch.stack(mask_video) for mask_video in mask])
    return mask.to(torch.int8)


In [18]:
# convert lists to tensors


In [20]:
#gc.collect()
train_np = np.array(X_train['points'].tolist(), dtype=np.float16)
train_seq = torch.tensor(train_np, dtype=torch.float16)

In [26]:
#gc.collect()
train_mask = create_attention_mask_from_points(train_seq)
print(train_mask.shape)

torch.Size([300, 2537])


In [21]:
train_y = tokens_label_train['input_ids']

In [ ]:
globals().pop("X_train", None)
globals().pop("train_np", None)
globals().pop("y_train", None)
globals().pop("tokens_label_train", None)
gc.collect()

In [23]:
gc.collect()
val_np = np.array(X_val['points'].tolist(), dtype=np.float16)
val_seq = torch.tensor(val_np, dtype=torch.float16)

In [24]:
gc.collect()
val_mask = create_attention_mask_from_points(val_seq)

In [25]:
val_y = tokens_label_val['input_ids']

In [ ]:
globals().pop("X_val", None)
globals().pop("val_np", None)
globals().pop("y_val", None)
globals().pop("tokens_label_val", None)
gc.collect()

In [27]:
gc.collect()
test_np = np.array(X_test['points'].tolist(), dtype=np.float16)
test_seq = torch.tensor(test_np, dtype=torch.float16)

In [28]:
gc.collect()
test_mask = create_attention_mask_from_points(test_seq)

In [29]:
test_y = tokens_label_test['input_ids']

In [ ]:
globals().pop("X_test", None)
globals().pop("test_np", None)
globals().pop("y_test", None)
globals().pop("tokens_label_test", None)
gc.collect()

<h1 style="font-size:200%; font-family:cursive; color:white;">6. Data Loader</h1>

In [ ]:

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)
globals().pop("train_seq", None)
globals().pop("train_mask", None)
globals().pop("train_y", None)
gc.collect()
# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
globals().pop("train_data", None)
globals().pop("train_sampler", None)
gc.collect()

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)
globals().pop("val_seq", None)
globals().pop("val_mask", None)
globals().pop("val_y", None)
gc.collect()

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)
globals().pop("val_data", None)
globals().pop("val_sampler", None)
gc.collect()

<h1 style="font-size:200%; font-family:cursive; color:white;">7. Model Architecture</h1>

In [ ]:
vocab_size = tokenizer.vocab_size
class BERT_Arch(nn.Module):

    def __init__(self, bert, vocab_size):
        super(BERT_Arch, self).__init__()
        
        self.bert = bert 
        
        # Dropout layer
        self.dropout = nn.Dropout(0.1)
        
        # ReLU activation function
        self.relu = nn.ReLU()

        # Dense layer 1
        self.fc1 = nn.Linear(768, 512)
        
        # Dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512, vocab_size)  # Use tokenizer's vocab size

        # Softmax activation function
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, seq_input, mask):
        outputs = self.bert(seq_input, attention_mask=mask, return_dict=False)
        
        # Use the last hidden state for each token (outputs[0])
        x = self.fc1(outputs[0])  # (batch_size, sequence_length, 512)

        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)  # (batch_size, sequence_length, vocab_size)
        
        # Apply softmax activation (for each token in the sequence)
        x = self.softmax(x)

        return x
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert, vocab_size)

# push the model to GPU
model = model.to(device)
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(),lr = 1e-5) 
# Define the loss function
cross_entropy = nn.CrossEntropyLoss()

# number of training epochs
epochs = 10

<h1 style="font-size:200%; font-family:cursive; color:white;">8. Fine - Tune</h1>

In [33]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Define the BLEU score function for a single prediction and reference
def calculate_bleu(preds, references):
    smoothie = SmoothingFunction().method4  # Smoothing method to avoid zero scores for short sentences
    score = sentence_bleu([references], preds, smoothing_function=smoothie)
    return score

# Function to convert token ids back to words (predictions and labels)
def decode_predictions(predictions, tokenizer):
    # Convert token IDs to tokens (words) using the tokenizer's decode method
    return [tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions]

In [34]:
def train():
    # Establecer el modelo en modo de entrenamiento
    model.train()
    
    # Inicializar acumuladores para la pérdida, predicciones y el BLEU score
    total_loss = 0
    total_preds = []
    total_bleu_score = 0
    
    # Iterar sobre los lotes en el DataLoader
    for step, batch in enumerate(train_dataloader):
        
        # Progreso cada 50 lotes
        if step % 50 == 0 and not step == 0:
            print(f'  Batch {step:>5,} of {len(train_dataloader):>5,}.')

        # Pasar el lote al dispositivo (GPU o CPU)
        sent_id, mask, labels = [r.to(device) for r in batch]
        print(sent_id.shape, mask.shape, labels.shape)
        # Limpiar los gradientes previamente calculados
        model.zero_grad()

        # Obtener las predicciones del modelo
        preds = model(sent_id, mask)
        
        # Calcular la pérdida entre los valores reales y los predichos
        loss = cross_entropy(preds, labels)

        # Acumular la pérdida total
        total_loss += loss.item()

        # Realizar retropropagación para calcular los gradientes
        loss.backward()

        # Clipping de los gradientes para evitar explosión de gradientes
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Actualizar los parámetros del modelo
        optimizer.step()

        # Mover las predicciones a CPU para procesamiento adicional
        preds = preds.detach().cpu().numpy()

        # Almacenar las predicciones para uso posterior
        total_preds.append(preds)

        # Decodificar predicciones y etiquetas en secuencias legibles
        decoded_preds = decode_predictions(preds.argmax(axis=-1), tokenizer)
        decoded_labels = decode_predictions(labels.cpu().numpy(), tokenizer)

        # Calcular el BLEU score para este lote
        batch_bleu_score = sum([calculate_bleu(pred, ref) for pred, ref in zip(decoded_preds, decoded_labels)]) / len(decoded_preds)
        total_bleu_score += batch_bleu_score

    # Calcular la pérdida promedio para la época
    avg_loss = total_loss / len(train_dataloader)

    # Concatenar todas las predicciones
    total_preds = np.concatenate(total_preds, axis=0)

    # Calcular el BLEU score promedio para la época
    avg_bleu_score = total_bleu_score / len(train_dataloader)

    # Devolver la pérdida promedio y el BLEU score promedio
    return avg_loss, avg_bleu_score

In [35]:
def evaluate():
    print("\nEvaluating...")

    # Establecer el modelo en modo de evaluación (desactiva dropout)
    model.eval()

    # Inicializar acumuladores para la pérdida y el BLEU score
    total_loss = 0
    total_bleu_score = 0
    total_preds = []

    # Tiempo inicial para mostrar el progreso
    t0 = time.time()

    # Iterar sobre los lotes en el DataLoader de validación
    for step, batch in enumerate(val_dataloader):
        
        # Progreso cada 50 lotes
        if step % 50 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f'  Batch {step:>5,}  of  {len(val_dataloader):>5,}.    Elapsed: {elapsed}.')
        
        # Pasar el lote al dispositivo (GPU o CPU)
        sent_id, mask, labels = [t.to(device) for t in batch]
        # Desactivar autograd para la evaluación
        with torch.no_grad():
            # Predicciones del modelo
            preds = model(sent_id, mask)

            # Calcular la pérdida de validación
            loss = cross_entropy(preds, labels)
            total_loss += loss.item()

            # Mover las predicciones a la CPU para procesamiento adicional
            preds = preds.detach().cpu().numpy()

            # Almacenar las predicciones para uso posterior
            total_preds.append(preds)

            # Decodificar predicciones y etiquetas en secuencias legibles
            decoded_preds = decode_predictions(preds.argmax(axis=-1), tokenizer)
            decoded_labels = decode_predictions(labels.cpu().numpy(), tokenizer)

            # Calcular el BLEU score para este lote
            batch_bleu_score = sum([calculate_bleu(pred, ref) for pred, ref in zip(decoded_preds, decoded_labels)]) / len(decoded_preds)
            total_bleu_score += batch_bleu_score

    # Calcular la pérdida promedio de validación
    avg_loss = total_loss / len(val_dataloader)

    # Calcular el BLEU score promedio
    avg_bleu_score = total_bleu_score / len(val_dataloader)

    # Concatenar todas las predicciones
    total_preds = np.concatenate(total_preds, axis=0)

    # Devolver la pérdida promedio y el BLEU score promedio
    return avg_loss, avg_bleu_score


In [ ]:
# set initial loss and BLEU score to infinite/zero
best_valid_loss = float('inf')
best_bleu_score = 0

# empty lists to store training and validation loss and BLEU score of each epoch
train_losses = []
valid_losses = []
train_bleu_scores = []
valid_bleu_scores = []

# for each epoch
for epoch in range(epochs):
    
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    # train model
    train_loss, train_bleu = train()
    
    # evaluate model
    valid_loss, valid_bleu = evaluate()
    
    # save the best model based on validation loss
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss and BLEU score
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    train_bleu_scores.append(train_bleu)
    valid_bleu_scores.append(valid_bleu)
    
    # print training and validation statistics
    print(f'\nTraining Loss: {train_loss:.3f} | Training BLEU: {train_bleu:.3f}')
    print(f'Validation Loss: {valid_loss:.3f} | Validation BLEU: {valid_bleu:.3f}')

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<h1 style="font-size:200%; font-family:cursive; color:white;">9. Make Predictions</h1>

In [ ]:
# get predictions for test data
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()

In [ ]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))